### Libraries

In [55]:
import torch
import copy
from torchvision import datasets, transforms
import numpy as np
import pickle
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt

# estimated time : 1m 23s


### Global Params

In [56]:
num_iter = 10
batch_size = 128
learning_rates = [0.005, .001, .0005, .0001]

### Import Data

In [57]:
def one_hot(y, nclass):
    n = y.size
    Y = np.zeros((n,nclass))
    Y[np.arange(n),y] = 1

    return Y

def train_test_split(X,y,ratio=0.2,shuffle = True):

    m = X.shape[0]
    if shuffle==True:
        index = np.arange(m)
        np.random.shuffle(index)
        X = X[index]
        y = y[index]

    split = np.round((1-ratio)*m).astype(int)

    X_train = X[:split]
    y_train = y[:split]

    X_test = X[split:]
    y_test = y[split:]

    return X_train, y_train,  X_test, y_test

transform = transforms.ToTensor()


# load the training dataset
train_dataset = datasets.FashionMNIST(root='./data', train =True, transform = transform, download=True)

#load the test dataset
test_dataset = datasets.FashionMNIST(root='./data', train =False, transform = transform, download=True)

# Using list comprehension to convert images and labels into tensors
x_train = torch.stack([image for image, label in train_dataset])  # Stack all images
y_train = torch.tensor([label for _, label in train_dataset])     # Create a tensor for labels

x_test = torch.stack([image for image, label in test_dataset])  # Stack all images
y_test = torch.tensor([label for _, label in test_dataset])     # Create a tensor for labels

# Convert to numpy arrays
x_train = x_train.numpy()  # Shape will be (60000, 1, 28, 28)
y_train = y_train.numpy()  # Shape will be (60000,)

x_test = x_test.numpy() # shape (10000, 1, 28, 28)
y_test = y_test.numpy()

x_train = x_train.reshape(x_train.shape[0], -1)  # Flatten to shape (60000, 784)
x_test = x_test.reshape(x_test.shape[0], -1)  # Flatten to shape (10000, 784)


X = x_train
Y = y_train

Y = one_hot(Y, len(np.unique(Y)))
y_test = one_hot(y_test, len(np.unique(y_test)))

X, Y, x_validation, y_validation = train_test_split(X, Y)

print(f'x.shape: {X.shape} y.shape: {Y.shape}')
print(f'x_validation.shape: {x_validation.shape} y_validation.shape: {y_validation.shape}')
print(f'x_test.shape: {x_test.shape} y_test.shape: {y_test.shape}')



# estimated time : 29s

x.shape: (48000, 784) y.shape: (48000, 10)
x_validation.shape: (12000, 784) y_validation.shape: (12000, 10)
x_test.shape: (10000, 784) y_test.shape: (10000, 10)


### Batch Normalization

Batch Normalization layer that scales and shifts normalized input.
The scaling w and shift b are learned by the network.

In [58]:
class BatchNorm:

    def __init__(self, input_shape, momentum=0.9, epsilon=1e-7):
        """
        Initializes the BatchNorm layer.

        Args:
            input_shape (tuple): Shape of the input vector, excluding batch size.
            momentum (float, optional): Momentum for running mean/variance. Defaults to 0.9.
            epsilon (float, optional): Small constant for numerical stability. Defaults to 1e-7.
        """
        d = input_shape

        # Learnable parameters (scale and shift)
        self.w = np.ones((1, d))  # Scale parameter (initialized to 1)
        self.b = np.zeros((1, d))  # Shift parameter (initialized to 0)

        # Gradients for scale and shift parameters
        self.dW = np.zeros((1, d))
        self.db = np.zeros((1, d))
        self.cache = None

        # Store running mean and variance for inference
        self.running_mean = np.zeros((1, d))
        self.running_var = np.ones((1, d))
        self.momentum = momentum
        self.epsilon = epsilon

    def forward(self, x, train=True):


        if train:
            # Calculate batch mean and variance
            mu = np.mean(x, axis=0, keepdims=True)
            var = np.mean((x - mu)**2, axis=0, keepdims=True)

            # Normalize the batch
            # (x - mu) / sqrt(var + epsilon)
            xmu = x - mu
            sqrtvar = np.sqrt(var + self.epsilon)
            ivar = 1.0 / sqrtvar
            xhat = xmu * ivar

            # Update running mean and variance
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mu
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var

            # Store intermediate values for backpropagation
            self.cache = (xhat, xmu, ivar, sqrtvar, var, mu)
        else:
            # Use running mean and variance during inference
            xhat = (x - self.running_mean) / np.sqrt(self.running_var + self.epsilon)

        # Scale and shift
        # w * xhat + b
        out = self.w * xhat + self.b


        return out

    def backward(self, dout):

        # Unfold the variables stored in cache during forward pass
        xhat, xmu, ivar, sqrtvar, var, mu = self.cache

        N, D = dout.shape

        # Gradient of scale and shift parameters

        # dL / dBeta = ∑ dL/dy
        self.db = np.sum(dout, axis=0, keepdims=True)

        # dL / dGamma = ∑ dL/dy * x_hat
        self.dW = np.sum(dout * xhat, axis=0, keepdims=True)

        # Gradient of normalized input

        # dL / dx_hat = dL/dy * gamma
        dxhat = dout * self.w

        # Gradients for batch normalization layer
        # dL / dVar = ∑ dL/dx_hat * (x - mean) * -1/2 * (var + epsilon)^(-3/2)
        dvar = np.sum(dxhat * xmu, axis=0) * -0.5 * (var + self.epsilon)**-1.5

        # dL / dMean = ∑ dL/dx_hat * -1/sqrt(var + epsilon) + dL/dVar * ∑ -2(x - mean) / N
        dmu = np.sum(dxhat * -ivar, axis=0) + dvar * np.sum(-2.0 * xmu, axis=0) / N

        dx1 = dxhat * ivar
        dx2 = dvar * 2.0 * xmu / N
        dZ = dx1 + dx2 + dmu / N

        return dZ


### Adam optimizer

In [59]:
class Adam:
    """
    Our favourite Adam optimization that combines momentum and rmsprop
    """
    def __init__(self, net, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        # params
        params = net.get_weights()
        self.learning_rate = learning_rate

        self.beta1=beta1
        self.beta2=beta2
        self.epsilon=epsilon

        # init
        self.VW = {}
        self.Vb = {}
        self.SW = {}
        self.Sb = {}
        for k in params.keys():
            self.VW[k] = 0
            self.Vb[k] = 0
            self.SW[k] = 0
            self.Sb[k] = 0
    def update(self, net):

        params = net.get_weights()
        dparams = net.get_dweights()

        beta1 = self.beta1
        beta2 = self.beta2

        for k,(dW,db) in dparams.items():
            W,b = params[k]
            # momentum
            self.VW[k] = beta1*self.VW[k] + (1.-beta1)*dW #dW
            self.Vb[k] = beta1*self.Vb[k] + (1.-beta1)*db #db
            # rmsprop
            self.SW[k] = beta2*self.SW[k] + (1.-beta2)*(dW**2) #dW**2
            self.Sb[k] = beta2*self.Sb[k] + (1.-beta2)*(db**2) #db**2

            W -= self.learning_rate * self.VW[k]/(np.sqrt(self.SW[k]) + self.epsilon) # W
            b -= self.learning_rate * self.Vb[k]/(np.sqrt(self.Sb[k]) + self.epsilon) # b

### Relu

During backpropagation, relU propagates the gradient only where x > 0

Because, the gradient of Relu is 1 for positive and 0 for negative inputs

In [60]:
class RelU:
    def __init__(self):
        self.input = None
    def forward(self, x):
        self.input = x
        return np.maximum(0,x)

    def backward(self, dout):

        return dout * (self.input > 0)



### Dropout

Randomly setting some neurons to zero during training

In [61]:
class Dropout:
    def __init__(self, p=0.5):
        self.probability = p
        self.mask = None

    def forward(self, x, train=True):

        if train:
            # Create a mask with the same shape as the input,
            # and set neurons to zero with probability
            self.mask = (np.random.rand(*x.shape) < self.probability) / self.probability
            return x * self.mask
        else:
            return x

    def backward(self, dout):

        # gradient is zero for the neurons that were turned off during forward pass
        return dout * self.mask

### SoftMax With Cross-Entropy loss

In [62]:
class Loss:
    def __init__(self):
        self.y = None
        self.y_pred = None

    def softmax(self, x):
        '''
            x: np.ndarray
        '''
        exps = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exps / np.sum(exps, axis=-1, keepdims=True)

    def cross_entropy(self, y, y_pred):
        '''
            y: np.ndarray
            y_pred: np.ndarray
        '''
        # Cross entropy loss
        # calculate the softmax of the predicted output
        y_pred = self.softmax(y_pred)
        # calculate the loss
        loss = -np.sum(y * np.log(y_pred + 1e-8), axis=-1)
        return loss

    def forward(self, y, y_pred):
        '''
            y: np.ndarray , the true labels
            y_pred: np.ndarray, the predicted labels
        '''
        self.y = y
        self.y_pred = y_pred
        return np.mean(self.cross_entropy(y, y_pred))

    def backward(self):
        '''
            returns the gradient of the loss with respect to the input
        '''
        return (self.softmax(self.y_pred) - self.y) / self.y.shape[0]


### Dense Layer
A fully connected layer :: y = Wx + b
    

In [63]:
class DenseLayer:
    def __init__(self, input_size, output_size):
        '''
            input_size: int
            output_size: int
        '''
        # he initialization
        self.w = np.random.randn(input_size, output_size) * np.sqrt(2/input_size)
        self.b = np.zeros((1, output_size))

        self.dW = np.zeros_like(self.w)
        self.db = np.zeros_like(self.b)

    def forward(self, x):
        '''
            x: np.ndarray
        '''
        self.x = x

        # Debugging: Print shapes to verify compatibility
        # print(f"Input shape: {x.shape}, Weights shape: {self.w.shape}, Bias shape: {self.b.shape}")

        return (x @ self.w) + self.b

    def backward(self, dout):
        '''
            dout: np.ndarray
        '''
        self.dW = self.x.T @ dout
        self.db = np.sum(dout, axis=0)
        return dout @ self.w.T


### FNN
Fully Connected Feed Forward Neural network

In [64]:
class FNN:
    def __init__(self):

# ############### Model 1 ################
#         # input layer
#         self.layer1 = DenseLayer(784, 128)
#         self.relu = RelU()
#         self.bn = BatchNorm(128)
#         self.drop = Dropout()

#         # Second layer
#         self.layer3 = DenseLayer(128, 64)
#         self.relu3 = RelU()
#         self.bn3 = BatchNorm(64)
#         self.drop3 = Dropout()

#         # output layer
#         self.layer4 = DenseLayer(64, 10)
#         self.softmax = Loss()

#         self.layers = {
#             'l1' : self.layer1,
#             'rel1' : self.relu,
#             'bn1' : self.bn,
#             'drp1' : self.drop,
#             'l3' : self.layer3,
#             'rel3': self.relu3,
#             'bn3': self.bn3,
#             'l4' : self.layer4

#         }

############### Model 2 ################
        # input layer
        self.layer1 = DenseLayer(784, 512)
        self.relu = RelU()
        self.bn = BatchNorm(512)
        self.drop = Dropout()

        # second layer
        self.layer2 = DenseLayer(512, 128)
        self.relu2 = RelU()
        self.bn2 = BatchNorm(128)
        self.drop2 = Dropout()

        # third layer
        self.layer3 = DenseLayer(128, 64)
        self.relu3 = RelU()
        self.bn3 = BatchNorm(64)
        self.drop3 = Dropout()

        # output layer
        self.layer4 = DenseLayer(64, 10)
        self.softmax = Loss()

        self.layers = {
            'l1' : self.layer1,
            'rel1' : self.relu,
            'bn1' : self.bn,
            'drp1' : self.drop,
            'l2': self.layer2,
            'rel2': self.relu2,
            'bn2': self.bn2,
            'drp2': self.drop2,
            'l3' : self.layer3,
            'rel3': self.relu3,
            'bn3': self.bn3,
            'l4' : self.layer4

        }

# ############### Model 3 ################
#         # input layer
#         self.layer1 = DenseLayer(784, 512)
#         self.relu = RelU()
#         self.bn = BatchNorm(512)
#         self.drop = Dropout()

#         # second layer
#         self.layer2 = DenseLayer(512, 128)
#         self.relu2 = RelU()
#         self.bn2 = BatchNorm(128)
#         self.drop2 = Dropout()

#         # third layer
#         self.layer3 = DenseLayer(128, 64)
#         self.relu3 = RelU()
#         self.bn3 = BatchNorm(64)
#         self.drop3 = Dropout()

#         # fourth layer
#         self.layer4 = DenseLayer(64, 32)
#         self.relu4 = RelU()
#         self.bn4 = BatchNorm(32)
#         self.drop4 = Dropout()

#         # output layer
#         self.layer5 = DenseLayer(32, 10)
#         self.softmax = Loss()

#         self.layers = {
#             'l1' : self.layer1,
#             'rel1' : self.relu,
#             'bn1' : self.bn,
#             'drp1' : self.drop,
#             'l2': self.layer2,
#             'rel2': self.relu2,
#             'bn2': self.bn2,
#             'drp2': self.drop2,
#             'l3' : self.layer3,
#             'rel3': self.relu3,
#             'bn3': self.bn3,
#             'drp3': self.drop3,
#             'l4' : self.layer4,
#             'rel4': self.relu4,
#             'bn4': self.bn4,
#             'drp4': self.drop4,
#             'l5' : self.layer5

#         }

    def forward(self, X):

        # loop through the layers
        for layer in self.layers.values():
            X = layer.forward(X)

        return X

    def calculate_loss(self, y, y_pred):
        '''
            y: np.ndarray -- real values of the target

            y_pred: np.ndarray -- predicted values of the target received from the forward pass

            Returns:
                - float -- the loss
                - np.ndarray -- the gradient of the loss with respect to the input

        '''

        loss =  self.softmax.forward(y, y_pred)
        dz = self.softmax.backward()

        return loss, dz

    def backward(self, dz):
        '''
            dz: np.ndarray --- The gradient of the loss got from the calculate_loss function
        '''

        for layer in reversed(self.layers.values()):
            dz = layer.backward(dz)

        return dz

    def get_weights(self):
        '''
            returns weights to Adam optimizer for updates
        '''
        weights = {}
        for k,layer in self.layers.items():
            if hasattr(layer, 'w') and hasattr(layer, 'b'):
                weights[k] = (layer.w, layer.b)
        return weights

    def get_dweights(self):
        '''
            returns the gradients of the weights to Adam optimizer for updates
        '''
        dweights = {}
        for k,layer in self.layers.items():
            if hasattr(layer, 'dW') and hasattr(layer, 'db'):
                dweights[k] = (layer.dW, layer.db)
        return dweights


    def save_model(self, filename):
        '''
            filename: str -- the name of the file to save the model
        '''

        weights = self.get_weights()  # Retrieve only weights and biases
        with open(filename, 'wb') as f:
            pickle.dump(weights, f)



    def load_model(self, filename):
        '''
            filename: str -- the name of the file to load the model
        '''
        with open(filename, 'rb') as f:
            data = pickle.load(f)


        for k,layer in self.layers.items():
            if k in data:  # Ensure the layer exists in the weights dictionary
                    layer.w, layer.b = data[k]




### Load From pickle

In [65]:
# # Load the model
# net = FNN(X.shape[1:], out_size=Y.shape[1])
# net.load_model(f'model_1905025.pkl')

# optimizer = Adam(net, learning_rate=learning_rate)
# loss_calculator = Loss()

# print('Load completed')

### Evaluation of Model

In [66]:
def evaluate_model(net, X, Y, loss_calculator: Loss):
  """
    Parameters:
    - net: The trained neural network model.
    - X: Validation feature set.
    - Y: Validation target set.
    - loss_calculator: Loss function to be used for calculating loss.

    Returns:
    - validation_loss: The average loss on the validation set.
    - validation_accuracy: Accuracy on the validation set.
    - macro_f1_score: Macro-F1 score on the validation set.
  """

  A = net.forward(X)

  # Calculate loss  (true_labels, predicted_labels)
  loss = loss_calculator.forward(Y, A)
  y_true = np.argmax(Y, axis=1)
  y_pred = np.argmax(A, axis=1)

  # Calculate validation accuracy
  validation_accuracy = np.sum(y_pred == y_true) / len(y_true)
  macro_f1_score = f1_score(y_true, y_pred, average='macro')

  return loss, validation_accuracy, macro_f1_score

### Plot the graphs

In [67]:
def plot_graph(data_list, lr, param):
    """
    Plots a graph for a given metric.

    Args:
        data_list (list): List of dictionaries with 'iteration' and 'value' for a specific parameter.
        lr (float): Learning rate used during training.
        param (str): Parameter name to display on the plot.
    """
    # Extract iterations and values from data_list
    iterations = [item['iteration'] for item in data_list]
    values = [item['value'] for item in data_list]

    # Plotting the data
    plt.figure(figsize=(10, 6))
    plt.plot(iterations, values, label=f'{param} (lr={lr})')
    plt.xlabel('Iteration')
    plt.ylabel(param)
    plt.title(f'{param.capitalize()} vs Iteration (Learning Rate: {lr})')
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_combined_graphs(data_dict, param):
    """
    Plots combined graphs for a given parameter across different learning rates.

    Args:
        data_dict (dict): Dictionary where each key is a learning rate and each value
                          is a list of dictionaries with 'iteration' and 'value'.
        param (str): Parameter name to display on the plot.
    """
    plt.figure(figsize=(10, 6))

    for lr, data_list in data_dict.items():
        # Extract iterations and values for each learning rate
        iterations = [item['iteration'] for item in data_list]
        values = [item['value'] for item in data_list]

        # Plot each learning rate series with a label
        plt.plot(iterations, values, label=f'lr={lr}')

    # Add labels, title, and legend
    plt.xlabel('Iteration')
    plt.ylabel(param)
    plt.title(f'{param.capitalize()} vs Iteration for Different Learning Rates')
    plt.legend()
    plt.grid(True)
    plt.show()



### Train FNN

In [68]:
np.random.seed(12)

num_batch = len(X)//batch_size
m = num_batch*batch_size   # apply cutoff


combined_train_loss = {}
combined_train_accuracy = {}
combined_val_loss = {}
combined_val_accuracy = {}
combined_val_f1 = {}

best_model = None
best_f1 = 0


for learning_rate in learning_rates:
  # Train mini batch
  net = FNN()

  optimizer = Adam(net,learning_rate=learning_rate)

  loss_calculator = Loss()


  # create a list of pairs
  train_loss_list = []
  train_accuracy_list = []
  val_loss_list = []
  val_accuracy_list = []
  val_f1_list = []


  for i in range(num_iter):

      permutation = np.random.permutation(m)
      train_accuracy = 0
      train_loss = 0

      for j in range(0,m,batch_size):

          indices = permutation[j:j+batch_size]
          X_batch, Y_batch = X[indices], Y[indices]

          A = net.forward(X_batch)

          loss, dZ  = net.calculate_loss(Y_batch, A)

          dZ = net.backward(dZ)

          optimizer.update(net)

          train_loss += loss
          train_accuracy += np.sum(np.argmax(A,axis=1)==np.argmax(Y_batch,axis=1))/len(X_batch)

      train_loss /= num_batch
      train_accuracy /= num_batch

      val_loss, val_accu, val_f1 = evaluate_model(net, x_validation, y_validation, loss_calculator)

      # update the best model and f1 score
      if val_f1 > best_f1:
          best_f1 = val_f1
          best_model = copy.deepcopy(net)

      print(f'Learning_rate:{learning_rate} iteration:{i+1} train_loss: {train_loss:.4f} train_accuracy: {train_accuracy:.4f} val_loss: {val_loss:.4f} val_accuracy: {val_accu:.4f} val_f1: {val_f1:.4f}')

      # Store metrics as {iteration, value} pairs
      train_loss_list.append({'iteration': i+1, 'value': train_loss})
      train_accuracy_list.append({'iteration': i+1, 'value': train_accuracy})
      val_loss_list.append({'iteration': i+1, 'value': val_loss})
      val_accuracy_list.append({'iteration': i+1, 'value': val_accu})
      val_f1_list.append({'iteration': i+1, 'value': val_f1})

  # Save lists in combined dictionaries with learning rate as the key
  combined_train_loss[learning_rate] = train_loss_list
  combined_train_accuracy[learning_rate] = train_accuracy_list
  combined_val_loss[learning_rate] = val_loss_list
  combined_val_accuracy[learning_rate] = val_accuracy_list
  combined_val_f1[learning_rate] = val_f1_list
  # plot_graph(train_loss_list, learning_rate, 'train_loss')
  # plot_graph(train_accuracy_list, learning_rate, 'train_accuracy')
  # plot_graph(val_loss_list, learning_rate, 'val_loss')
  # plot_graph(val_accuracy_list, learning_rate, 'val_accuracy')
  # plot_graph(val_f1_list, learning_rate, 'val_f1')


# # Plot combined graphs for each metric
# plot_combined_graphs(combined_train_loss, 'train_loss')
# plot_combined_graphs(combined_train_accuracy, 'train_accuracy')
# plot_combined_graphs(combined_val_loss, 'val_loss')
# plot_combined_graphs(combined_val_accuracy, 'val_accuracy')
# plot_combined_graphs(combined_val_f1, 'val_f1')




Learning_rate:0.005 iteration:1 train_loss: 0.6292 train_accuracy: 0.7683 val_loss: 0.5206 val_accuracy: 0.8115 val_f1: 0.8063
Learning_rate:0.005 iteration:2 train_loss: 0.4754 train_accuracy: 0.8288 val_loss: 0.4649 val_accuracy: 0.8343 val_f1: 0.8337
Learning_rate:0.005 iteration:3 train_loss: 0.4448 train_accuracy: 0.8400 val_loss: 0.4432 val_accuracy: 0.8406 val_f1: 0.8389
Learning_rate:0.005 iteration:4 train_loss: 0.4208 train_accuracy: 0.8470 val_loss: 0.4385 val_accuracy: 0.8433 val_f1: 0.8439
Learning_rate:0.005 iteration:5 train_loss: 0.4126 train_accuracy: 0.8514 val_loss: 0.4246 val_accuracy: 0.8514 val_f1: 0.8507
Learning_rate:0.005 iteration:6 train_loss: 0.3975 train_accuracy: 0.8565 val_loss: 0.4256 val_accuracy: 0.8492 val_f1: 0.8490
Learning_rate:0.005 iteration:7 train_loss: 0.3857 train_accuracy: 0.8612 val_loss: 0.4152 val_accuracy: 0.8509 val_f1: 0.8509
Learning_rate:0.005 iteration:8 train_loss: 0.3807 train_accuracy: 0.8625 val_loss: 0.4100 val_accuracy: 0.8562

### Test FNN

In [69]:
test_loss, test_accu, test_f1 = evaluate_model(best_model, x_test, y_test, loss_calculator)
print(f'test_loss: {test_loss:.4f} test_accuracy: {test_accu:.4f} test_f1: {test_f1:.4f}')

# estimated time : 7.6s

test_loss: 0.4180 test_accuracy: 0.8531 test_f1: 0.8523


### Get the confusion matrix


In [70]:
'''
confusion matrix: significance
    - The diagonal elements represent the number of points for which the predicted label
        is equal to the true label
    - The off-diagonal elements are those that are mislabeled by the classifier
'''

y_pred = best_model.forward(x_validation)
confusion_mat = confusion_matrix(np.argmax(y_validation, axis=1), np.argmax(y_pred, axis=1))
print(f'Confusion matrix for this model: \n{confusion_mat}')

Confusion matrix for this model: 
[[ 953    4   27   76    4    2  141    0   11    0]
 [   4 1150    3   20    2    0    0    0    1    0]
 [  10    7  953   13  177    0   86    0    6    0]
 [  15   14   16 1064   48    0   21    0    1    0]
 [   2    6   85   44 1011    0   58    0    7    0]
 [   0    0    0    1    0 1097    1   60    3   19]
 [ 149    3  136   59  114    0  741    0   15    0]
 [   0    0    0    0    0   26    0 1104    0   29]
 [   3    3   10    8    9    5   13    5 1125    2]
 [   0    1    0    0    0   16    0   89    0 1112]]


### Store in pickle

In [71]:
weights = best_model.get_weights()
# save weights in a pickle file
with open('model_1905025.pkl', 'wb') as f:
    pickle.dump(weights, f)
